## Neural Networks
In this notebook, we will walk through one main neural nets example. And that is, classifying the infamous MNIST dataset. **If you have no experience with neural nets prior to this notebook, I recommend doing a quick search for an "intro to neural nets"**, there are multiple tutorials/blog posts out there and you can choose the one that works for you.

Here, we will use the `Flux` package, but if you want to look at other packages I encourage you to look at `Knet.jl` and `TensorFlow.jl`.

In [1]:
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated
using Images

LoadError: ArgumentError: Package Flux [587475ba-b771-5e3f-ad9e-33799f191a9c] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


Let's take a look at one of the images.

In [2]:
imgs = MNIST.images()
colorview(Gray, imgs[100])

LoadError: UndefVarError: MNIST not defined

In [3]:
typeof(imgs[3])

LoadError: UndefVarError: imgs not defined

First, we will transofrm the gray scale values to Float32 types. Here, using Float32 will speedup the neural network substantially withough compromising the quality of the solution.

In [4]:
myFloat32(X) = Float32.(X)
fpt_imgs = myFloat32.(imgs) 

LoadError: UndefVarError: imgs not defined

In [5]:
typeof(fpt_imgs[3])

LoadError: UndefVarError: fpt_imgs not defined

We will now create a few helpful functions...

In [6]:
vectorize(x) = x[:]
vectorized_imgs = vectorize.(fpt_imgs);

LoadError: UndefVarError: fpt_imgs not defined

In [7]:
typeof(vectorized_imgs)

LoadError: UndefVarError: vectorized_imgs not defined

We will again make use of `...` as the splat operator to concatenate all images into one matrix.

In [8]:
X = hcat(vectorized_imgs...)
size(X)

LoadError: UndefVarError: vectorized_imgs not defined

Now, every column in `X` is an image of a number. We have `60,000` images. When reshaped into a 28-by-28 matrix, and displayed as an image, can be seen as a handwritten number. Here is an example below.

In [9]:
onefigure = X[:,3]
t1 = reshape(onefigure,28,28)
colorview(Gray,t1)

LoadError: UndefVarError: X not defined

Next, we will obtain the labels. These are the true labels for the `60,000` images.

In [10]:
labels = MNIST.labels()
labels[1]

LoadError: UndefVarError: MNIST not defined

From these labels, we will create a new output column for each image. These columns will be indicator vectors of where the correct label is.

For example if the figure corresponding to column `X[:,i]` is a `3`, the `i'th` column in this new matrix `Y` is `[0 0 0 1 0 0 0 0 0 0]`. (It is the entry number 4 because entry 1 corresponds to the digit 0, so the counting starts from zero). The `onehotbatch` function allows us to create this easily.

In [11]:
Y = onehotbatch(labels, 0:9)

LoadError: UndefVarError: onehotbatch not defined

And now we will actually build our neural network. We will use two layers. The hidden layer will have 32 nodes, and the output layer will have 10 nodes. i.e. we will go from: `28*28 => 32 => 10`.

In [12]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

LoadError: UndefVarError: Dense not defined

What does `m`, the neural network mean here? 

If you've worked with neural networks before you know that the solution is often not found by just one pass on the neural network. One pass happens, and a solution is generated at the output layer, then this solution is compared to the ground truth solution we already have (the columns from `Y`), and the network goes back and adjusts its weights and parameters and then try again. Here, since `m` is not "trained" yet, one pass of `m` on a figure generates the following (not-so-great) answer. We will see later how this changes after training.

In [13]:
m(onefigure)

LoadError: UndefVarError: m not defined

To run our neural network, we need a loss function and an accuracy function. The accuracy function is used to compare the output result from the output layer in the neural network to the groundtruth result. The loss function is used to evaluate the performance of the overall model after new weights have been recalculated at each pass.

In [14]:
loss(x, y) = Flux.crossentropy(m(x), y)
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

accuracy (generic function with 1 method)

Finally, we will repeat our data so that we have more samples to pass to the neural network, which means there will be more chances for corrections.

In [15]:
datasetx = repeated((X, Y), 200)
C = collect(datasetx);

LoadError: UndefVarError: X not defined

We will create a function to display the loss at each step.

In [16]:
evalcb = () -> @show(loss(X, Y))

#1 (generic function with 1 method)

In [17]:
ps = Flux.params(m)

LoadError: UndefVarError: Flux not defined

Finally, we are ready to train the model, we will use the `Flux.train!` function. Let's take a look at the documentation.

In [18]:
?Flux.train!

UndefVarError: UndefVarError: Flux not defined

In [19]:
opt = ADAM()
Flux.train!(loss, ps, datasetx, opt, cb = throttle(evalcb, 10))

LoadError: UndefVarError: ADAM not defined

We will now get the test data.

In [20]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...);
test_image = m(tX[:,1])

LoadError: UndefVarError: MNIST not defined

In [21]:
argmax(test_image) - 1

LoadError: UndefVarError: test_image not defined

In [22]:
t1 = reshape(tX[:,1],28,28)
colorview(Gray, t1)

LoadError: UndefVarError: tX not defined

What about the image we tried a few cells earlier and returned the "not-so-great" answer.

In [23]:
onefigure = X[:,2]
m(onefigure)

LoadError: UndefVarError: X not defined

In [24]:
Y[:,2]

LoadError: UndefVarError: Y not defined

# Finally...
After finishing this notebook, you should be able to:
- [ ] prepare data to fit the format to create a neural network using Flux.jl
- [ ] create a neural network with Flux.jl
- [ ] creating an accuracy function and loss function to be passed to train the neural network
- [ ] train the neural network
- [ ] describe a few tips that can help make your nerual network faster or more accurate (such as using Float32 as opposed to Float32)